In [ ]:
from tsim.utils.encoder import ColorEncoder3
import stim
import tsim
import sinter
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# here we define the non-fault tolerant encoding circuit, with noise
# NOTE: since this circuit is not fault-tolerant, it will always reduce the circuit distance to 1
# As a hack, we only inserted DEPOLARIZE noise in circuit locations, where noise is correctable
# With the current (artificial) noise model, this circuit has distance 3


def make_encoding_circuit(p: float = 0.01):
    return f"""
    R 0 1 2 3 4 5
    TICK
    SQRT_Y_DAG 0 1 2 3 4 5
    DEPOLARIZE1({p}) 0 1
    TICK
    CZ 1 2 3 4 5 6
    DEPOLARIZE2({p}) 1 2
    TICK
    SQRT_Y 6
    DEPOLARIZE1({p}) 6
    TICK
    CZ 0 3 2 5 4 6
    TICK
    SQRT_Y 2 3 4 5 6
    DEPOLARIZE1({p}) 2 4 6
    TICK
    CZ 0 1 2 3 4 5
    TICK
    DEPOLARIZE1({p}) 0 1 2 3 4 5 6
    SQRT_Y 1 2 4
    TICK
    X 3  # <------- the encoding circuit in https://arxiv.org/html/2412.15165v1
    Z 5 1  #        is only defined up to Paulis. Here, we track these Paulis (although they don't affect detectors in STIM)
    DEPOLARIZE1({p}) 0 1 2 3 4 5 6
    TICK
    """

In [ ]:
circ = stim.Circuit(make_encoding_circuit())

In [ ]:
tsim.Circuit(make_encoding_circuit()).diagram(height=250)

In [ ]:
# We can verify this has the correct stabilizers by looking at the flow generators
circ.flow_generators()

In [ ]:
# If we add Steane detectors and an observable, we can check that
# the current noise model has circuit distance 3
# using Stim's `shortest_graphlike_error` feature

circ_with_detectors = stim.Circuit(
    make_encoding_circuit()
    + """
    M 0 1 2 3 4 5 6
    DETECTOR rec[-7] rec[-6] rec[-5] rec[-4]
    DETECTOR rec[-6] rec[-5] rec[-3] rec[-2]
    DETECTOR rec[-5] rec[-4] rec[-3] rec[-1]
    OBSERVABLE_INCLUDE(0) rec[-7] rec[-6] rec[-2]
"""
)
print(len(circ_with_detectors.shortest_graphlike_error()))  # circuit distance

In [ ]:
# We use tsim's `ColorEncoder3` to build the circuit
# This works as follows:
# `encoder.initialize(circ)` will perform the gates in circ and then append encoding circuit to each qubit in circ
# `encoder.encode_transversally(circ)` will replace each gate in circ with a transversal logical gate.
#       If detectors are present, they will be turned into 3 detectors (one for each Steane stabilizer)
#       and the observable will be turned into a measurement of the logical observable


def make_memory_circuit(p: float = 0.01, num_rounds: int = 3):
    """
    Generate a Gemini-style memory circuit. Qubits are prepared in logical |+>/|0> states
    using above encoding circuit. There are no mid-circuit measurements.
    Therefore, we add 2 new code blocks per round.
    """
    encoder = ColorEncoder3()

    num_rounds = 3
    num_qubits = 2 * num_rounds + 1
    qubits = list(range(num_qubits))
    aux_z = qubits[1::2]
    aux_x = qubits[2::2]

    encoder.initialize(
        f"""
        RX {' '.join(map(str, aux_z))}
        R {' '.join(map(str, aux_x))}
        """,
        encoding_program_text=make_encoding_circuit(p),
    )
    encoder.encode_transversally("R 0")
    encoder.encode_transversally(
        f"""
        CNOT 0 1
        DEPOLARIZE2({p}) 0 1
        X_ERROR({p}) 1
        M 1
        DETECTOR rec[-1]
        CNOT 2 0
        DEPOLARIZE2({p}) 2 0
        Z_ERROR({p}) 2
        MX 2
        TICK
    """
    )
    for i in range(1, num_rounds):
        encoder.encode_transversally(
            f"""
        CNOT 0 {aux_z[i]}
        DEPOLARIZE2({p}) 0 {aux_z[i]}
        X_ERROR({p}) {aux_z[i]}
        M {aux_z[i]}
        DETECTOR rec[-1] rec[-3]
        CNOT {aux_x[i]} 0
        DEPOLARIZE2({p}) {aux_x[i]} 0
        Z_ERROR({p}) {aux_x[i]}
        MX {aux_x[i]}
        DETECTOR rec[-1] rec[-3]
        """
        )
    encoder.encode_transversally(
        f"""
        X_ERROR({p}) 0
        M 0
        DETECTOR rec[-1] rec[-3]
        OBSERVABLE_INCLUDE(0) rec[-1]
        """
    )

    return encoder.circuit


tsim_circuit = make_memory_circuit(p=0.01, num_rounds=3)
circ = tsim_circuit._stim_circ
tsim_circuit.diagram(height=800)

In [ ]:
# Check that the fault distance of the full memory circuit is 3
len(circ.shortest_graphlike_error())

In [ ]:
from tesseract_decoder.tesseract_sinter_compat import TesseractSinterDecoder
from ldpc.sinter_decoders.sinter_lsd_decoder import SinterLsdDecoder
from ldpc.sinter_decoders.sinter_bposd_decoder import SinterBpOsdDecoder
from relay_bp.stim import sinter_decoders
from mwpf import SinterMWPFDecoder

In [ ]:
bp_lsd_decoder = SinterLsdDecoder(
    max_iter=5,
    bp_method="minimum_sum",
    ms_scaling_factor=0.15,
    lsd_order=0,
)
bp_osd_decoder = SinterBpOsdDecoder(
    max_iter=100,
    bp_method="minimum_sum",
    ms_scaling_factor=0.15,
)
tesseract_dec = TesseractSinterDecoder()
ibm_decoder_dict = (
    sinter_decoders(  # these parameters are likely all wrong, need to be tuned
        gamma0=0.1,
        pre_iter=80,
        num_sets=60,
        set_max_iter=60,
        gamma_dist_interval=(-0.24, 0.66),
        stop_nconv=1,
    )
)
mwpf_decoder = SinterMWPFDecoder(cluster_node_limit=50)

In [ ]:
# use sinter for sampling and decoding
ps = np.logspace(-4, -2, 10)
rounds = 5

tasks = [
    sinter.Task(
        circuit=make_memory_circuit(p=p, num_rounds=rounds)._stim_circ,
        decoder=decoder_str,
        collection_options=sinter.CollectionOptions(
            max_shots=1_000_000, max_errors=2000
        ),
        json_metadata={"rounds": rounds, "p": p, "decoder": decoder_str},
    )
    for p in ps
    for decoder_str in [
        "bp_lsd",
        "pymatching",  # should fail since Steane code is not matchable / decoding graph is hypergraph
        "bp_osd",
        "tesseract",
        # "mem-bp",
        # "mwpf",  # slow
    ]
]

save_resume_filepath = "steane_decoding_data.csv"
stats = sinter.collect(
    tasks=tasks,
    num_workers=8,
    print_progress=False,
    custom_decoders={
        "bp_lsd": bp_lsd_decoder,
        "bp_osd": bp_osd_decoder,
        "tesseract": tesseract_dec,
        "mwpf": mwpf_decoder,
        **ibm_decoder_dict,
    },
    save_resume_filepath=save_resume_filepath,
)

In [ ]:
save_resume_filepath = "steane_decoding_data.csv"
stats = sinter.stats_from_csv_files(save_resume_filepath)
ax = plt.subplot()
sinter.plot_error_rate(
    ax=ax,
    stats=stats,
    x_func=lambda x: x.json_metadata["p"],
    group_func=lambda x: x.json_metadata["decoder"],
    failure_units_per_shot_func=lambda x: x.json_metadata["rounds"],
)
ax.set_xscale("log")
ax.set_yscale("log")
plt.plot(ps, ps, color="black", linestyle="--")
plt.plot(ps, np.array(ps) ** 2 * 1e2, color="gray", linestyle=":")
plt.xlabel("Physical error rate")
plt.ylabel("Logical error rate")
plt.title("Steane code, 5 rounds memory circuit, Steane style")
ax.legend();